In [2]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
from matplotlib.colors import TABLEAU_COLORS

def find_local_extrema(series):
    """Find local maxima and minima in a pandas Series."""
    local_maxima = (series.shift(1) < series) & (series.shift(-1) < series)
    local_minima = (series.shift(1) > series) & (series.shift(-1) > series)
    return local_maxima, local_minima

def split_dataframe(df):
    """Split dataframe into segments of monotonic increasing or decreasing 'eta'."""
    local_maxima, local_minima = find_local_extrema(df['eta'])
    
    # Get indices of local maxima and minima
    extrema_indices = np.sort(np.concatenate([local_maxima[local_maxima].index, local_minima[local_minima].index]))
    
    # Split dataframe into segments
    segments = []
    start_idx = 0
    for end_idx in extrema_indices:
        segments.append(df.iloc[start_idx:end_idx+1])
        start_idx = end_idx + 1
    
    # Append the last segment
    if start_idx < len(df):
        segments.append(df.iloc[start_idx:])
    
    return segments

%matplotlib qt
# Get the list of .csv files in the data/ folder
csv_files = [file for file in os.listdir('data/') if file.endswith('.csv')]

# Create an empty list to store the dataframes
dataframes = []
colors = list(TABLEAU_COLORS)

# Import each .csv file into a dataframe and append it to the list
for file in csv_files:
    if '2' in file: continue
    filepath = os.path.join('data/', file)
    df = pd.read_csv(filepath)
    dataframes.append(df)

# Plot all three dataframes
for i, df in enumerate(dataframes):
    segments = split_dataframe(df)
    for segment in segments:
        plt.plot(segment['eta'], segment['L2'], color=colors[i])
    # plt.plot(df['eta'], (df['L2']), label=f'{csv_files[i][:-4]}')    

plt.plot([-1.0, -0.25],[0.0, 0.0], color='black', linestyle='solid')
plt.plot([-0.25, 1.0],[0.0, 0.0], color='black', linestyle='dashed')

plt.xlim(-0.5, -0.2)
plt.ylim(-0.03, 0.55)
# plt.vlines(-0.41245, 0, 0.55, linestyles='dashed', colors='r')
# plt.legend()
plt.show()

In [36]:
list(TABLEAU_COLORS)

['tab:blue',
 'tab:orange',
 'tab:green',
 'tab:red',
 'tab:purple',
 'tab:brown',
 'tab:pink',
 'tab:gray',
 'tab:olive',
 'tab:cyan']

In [31]:
df = pd.read_csv(os.path.join('data/', 'LS_sym.csv'))
plt.plot(df['eta'])
plt.show()

In [33]:
segments = split_dataframe(df)
for segment in segments:
    plt.plot(segment['eta'], segment['L2'])
plt.show()

In [11]:
from findiff import FinDiff
dx = 0.25
d2_dx2 = FinDiff(0, dx, 2)
d_dx = FinDiff(0, dx, 1)

def free_energy(u, eta, dx=0.25):
    dF = - eta * u ** 2 - u ** 4 * 0.5 - d_dx(u) ** 2 + d2_dx2(u) ** 2
    return np.sum(dF) / len(dF)